In [ ]:
%%capture
!pip install  lightautoml
!pip install  pandas

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

pd.set_option('display.max_columns', None)
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']

import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

from drive.MyDrive.Kaggle.Imputation_062022.src.functions import *

In [ ]:
data = pd.read_csv('drive/MyDrive/Kaggle/Imputation_062022/src/data/data.csv', index_col='row_id')
submission = pd.read_csv('drive/MyDrive/Kaggle/Imputation_062022/src/data/sample_submission.csv', index_col='row-col')

In [ ]:
N_THREADS = 4
RANDOM_STATE = 21
# TEST_SIZE = 0.2
TIMEOUT = 0.5 * 4000
TARGET_NAME = 'value' # it will be changed to column
# N_FOLDS = 15

In [ ]:
set_seed(100)
col_list, F1, F2, F3, F4, missing_columns = get_lists(data)

In [ ]:
task = Task('reg', metric='mse', loss='mse')

In [ ]:
miss_columns = data.columns[data.isnull().any()]
sub = pd.DataFrame(columns=['row-col', 'value'])
data.reset_index(inplace=True)
for column in miss_columns:
    column_list=[x for x in miss_columns if (column[:3] in x)]
    train_data = data.loc[(data[column].notnull())][column_list + ['row_id']]
    test_data = data.loc[(data[column].isnull())][column_list + ['row_id']]

    roles = {'target': column,
             'drop': ['row_id']
             }
    
    automl = TabularAutoML(task=task,
                           timeout=TIMEOUT,
                           cpu_limit=N_THREADS,
                           general_params={'use_algos': [['lgb']]}
                          )

    oof_pred = automl.fit_predict(train_data, roles=roles, verbose=3)
    test_pred = automl.predict(test_data)

    sub = sub.append(pd.DataFrame({'row-col': test_data['row_id'].astype(str) + '-' + column,
                                   'value': test_pred.data[:, 0]}))

[20:23:55] Stdout logging level is INFO3.
[20:23:55] Task: reg

[20:23:55] Start automl preset with listed constraints:
[20:23:55] - time: 2000.00 seconds
[20:23:55] - CPU: 4 cores
[20:23:55] - memory: 16 GB

[20:23:55] Train data shape: (981603, 16)

[20:23:58] Feats was rejected during automatic roles guess: []
[20:23:58] Layer 1 train process start. Time left 1997.49 secs
[20:23:59] [1]	valid's l2: 1.00387
[20:23:59] Training until validation scores don't improve for 100 rounds
[20:24:03] [100]	valid's l2: 1.0042
[20:24:04] Selector_LightGBM fitting and predicting completed
[20:24:04] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[20:24:04] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[20:24:04] [1]	valid's l2: 1.00387
[20:24:04] Training until validation scores don't improve for 100 rounds
[20:24:09] [100]	valid's l2: 1.0042
[20:24:09] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[20:24:10] [1]	valid's l2: 0.998365
[20:24:10] Traini

In [ ]:
submission = make_submission(sub.set_index('row-col'), sample)
submission.to_csv('drive/MyDrive/Kaggle/Imputation_062022/submissions/LGM_autoML.csv')

100%|██████████| 1000000/1000000 [1:47:29<00:00, 155.05it/s]
